In [1]:
import numpy as np
from typing import Optional, Tuple

Zadanie rozgrzewkowe:
Napisać mnożenie macierzy w ulubionym języku programowania.

In [2]:
def agh_superfast_matrix_multiply(a: np.matrix, b: np.matrix) -> np.matrix:
    """Perform totally ordinary multiplication of matrices.
    
    :param a: matrix with dimensions n by m
    :param b: matrix with dimensions m by p
    :return:  matrix with dimensions n by p
    """
    n = a.shape[0]
    m_a = a.shape[1]
    m_b = b.shape[0]
    p = b.shape[1]
    
    if m_a != m_b:
        raise Exception('Matrices a and b cannot be multipled')
    
    c = np.zeros((n, p))
    
    for i in range(0, n):
        for j in range(0, p):
            for k in range(0, m_a):
                c[i, j] += a[i, k] * b[k, j]
    return c                

In [3]:
m1 = np.matrix([[1, 2],
                [3, 4],
                [4, 5],
                [5, 1]])

m2 = np.matrix([[1, 2, 3],
                [4, 5, 6]])

res = agh_superfast_matrix_multiply(m1, m2)
assert np.allclose(res, m1 * m2), "Wrong multiplication result"

Napisać kod do eliminacji Gaussa bez pivotingu:

In [4]:
def naive_gauss(a: np.matrix, b:np.matrix) -> np.matrix:
    n = a.shape[0]
    
    for k in range(0, n-1):
        for i in range(k+1, n):
            for j in range(k, n):
                a[i,j] -= (a[i,k]/a[k,k])*a[k,j]
            b[i] -= (a[i,k]/a[k,k])*b[k]
    
    x = np.zeros(n)
    for i in range(n-1,-1,-1):
        _sum = 0
        for j in range(i+1, n):
            _sum += a[i,j] * x[j]
        x[i] = (b[i] - _sum)/a[i,i]
        
    return np.matrix(x).transpose()


In [5]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()

x = naive_gauss(A, b)

np.allclose(np.dot(A, x), b)

True

Napisać kod do eliminacji Gaussa z pivotingiem:

In [6]:
def pivoting_gauss(a: np.matrix, b:np.matrix) -> np.matrix:
    n = a.shape[0]
    s = np.zeros(n)
    l = np.zeros(n, dtype=int)
    x = np.zeros(n)   
    
    r = 0
    rmax = 0
    smax = 0
    xmult = 0
    _sum = 0
    
    for i in range(0, n):
        l[i] = i
        smax = 0
        for j in range(0, n):
            smax = max(smax, abs(a[i,j]))
        s[i] = smax
        
    for k in range(0, n-1):
        rmax = 0
        for i in range(k, n):
            r = abs(a[l[i], k] / s[l[i]])
            if(r > rmax):
                rmax = r
                j = i
        tmp = l[j]
        l[j] = l[k]
        l[k] = tmp
        for i in range(k+1, n):
            xmult = a[l[i], k]/a[l[k], k]
            a[l[i], k] = xmult
            for j in range(k+1, n):
                a[l[i], j] = a[l[i], j] - (xmult)*a[l[k], j]
                
    for k in range(0, n):
        for i in range(k+1, n):
             b[l[i]] = b[l[i]] - a[l[i],k]*b[l[k]]
        
    x[n-1] = b[l[n-1]]/a[l[n-1], n-1]
        
    for i in range(n-1,-1,-1):
        _sum = b[l[i]]
        for j in range(i+1, n):
             _sum = _sum - a[l[i],j]*x[j]
        x[i] = _sum/a[l[i],i]
    
    return np.matrix(x).transpose()

In [7]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()

_x = np.linalg.solve(A, b)
x = pivoting_gauss(A, b)

np.allclose(_x, x)

True

Zaimplementować algorytm faktoryzacji LU macierzy

In [47]:
def agh_superfast_lu(a: np.matrix) -> Optional[Tuple[np.matrix, np.matrix]]:
    """Perform LU decomposition of a matrix.
    
    :param a: matrix n x n
    :return:  tuple(l, u): l -> lower diagonal matrix, u -> upper diagonal matrix
    """
    n = a.shape[0]
    l = np.zeros((n,n))
    u = np.zeros((n,n))
    
    for k in range(0, n):
        l[k,k] = 1
        for j in range(k, n):
            sum = 0
            for s in range(0, k):
                sum = sum + l[k,s]*u[s,j]
            u[k,j] = a[k,j] - sum
            
        for i in range(k+1, n):
            sum = 0
            for s in range(0, k):
                sum = sum + l[i,s]*u[s,k]
            if(u[k,k] != 0):
                l[i,k] = (a[i,k] - sum)/u[k,k]
            else:
                return None
                    
    return (l, u)
            

In [48]:
a = np.matrix([[5.0, 3.0, 2.0],
               [1.0, 2.0, 0.0],
               [3.0, 0.0, 4.0]])

In [53]:
x = agh_superfast_lu(a)
if(x != None):
    (l, u) = x
    print(np.allclose(a, agh_superfast_matrix_multiply(l, u)))
else:
    print("The matrix does not have the LU decomposition")

True


Zaimplementować algorytm faktoryzacji Cholesky'ego macierzy

In [76]:
def agh_superfast_cholesky(a: np.matrix) -> Optional[np.matrix]:
    """Perform a Cholesky decomposition of a matrix.
    
    :param a: matrix n x n
    :return: l: l*l^t = a, l:lower triangular with a positive diagonal
    """
    n = a.shape[0]
    l = np.zeros((n,n))
    
    for k in range(0, n):
        sum = 0
        for s in range(0, k):
            sum = sum + l[k,s]**2
        l[k,k] = (a[k,k] - sum)**(1/2)
        
        for i in range(k+1, n):
            sum = 0
            for s in range(0, k):
                sum = sum + l[i,s]*l[k,s]
            l[i,k] = (a[i,k] - sum)/l[k,k]
    
    return l

In [77]:
a = np.matrix([[4.0, 3.0, 2.0, 1.0],
               [3.0, 3.0, 2.0, 1.0],
               [2.0, 2.0, 2.0, 1.0],
               [1.0, 1.0, 1.0, 1.0]])

In [78]:
l = agh_superfast_cholesky(a)
print(l)
print(l.transpose())

[[2.         0.         0.         0.        ]
 [1.5        0.8660254  0.         0.        ]
 [1.         0.57735027 0.81649658 0.        ]
 [0.5        0.28867513 0.40824829 0.70710678]]
[[2.         1.5        1.         0.5       ]
 [0.         0.8660254  0.57735027 0.28867513]
 [0.         0.         0.81649658 0.40824829]
 [0.         0.         0.         0.70710678]]


In [79]:
print(np.allclose(a, agh_superfast_matrix_multiply(l, l.transpose())))

True
